In [1]:
!rm imdb_full.pkl
!rm imdb_word_index.json
!wget https://s3.amazonaws.com/text-datasets/imdb_full.pkl
!wget https://s3.amazonaws.com/text-datasets/imdb_word_index.json

--2019-10-05 19:47:42--  https://s3.amazonaws.com/text-datasets/imdb_full.pkl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65552540 (63M) [application/octet-stream]
Saving to: ‘imdb_full.pkl’

imdb_full.pkl       100%[===================>]  62.52M  83.4MB/s    in 0.7s    

2019-10-05 19:47:43 (83.4 MB/s) - ‘imdb_full.pkl’ saved [65552540/65552540]

--2019-10-05 19:47:43--  https://s3.amazonaws.com/text-datasets/imdb_word_index.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1641221 (1.6M) [application/octet-stream]
Saving to: ‘imdb_word_index.json’

imdb_word_index.jso 100%[===================>]   1.56M  --.-KB/s    in 0.08s   

2019-10-05 19:47:43 (20.3 MB/s) - ‘imdb_word_inde

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import pickle
data = pickle.load(open('imdb_full.pkl', 'rb'))

import json
vocab = json.load(open('imdb_word_index.json'))

In [3]:
inv = {idx:word for word, idx in vocab.items()}

(X_train, y_train), (Xt, yt) = data

trainidx = [i for i, x in enumerate(X_train) if len(x) < 400]
trainidx, devidx = train_test_split(trainidx, train_size=0.8, random_state=1378)
X = [X_train[i] for i in trainidx]
y = [y_train[i] for i in trainidx]

Xd = [X_train[i] for i in devidx]
yd = [y_train[i] for i in devidx]

testidx = [i for i, x in enumerate(Xt) if len(x) < 400]
testidx, remaining_idx =  train_test_split(testidx, train_size=0.2, random_state=1378)

Xt = [Xt[i] for i in testidx]
yt = [yt[i] for i in testidx]

def invert_and_join(X) :
    X = [[inv[x] for x in doc] for doc in X]
    X = [" ".join(x) for x in X]
    return X

X = invert_and_join(X)
Xd = invert_and_join(Xd)
Xt = invert_and_join(Xt)

texts = {'train' : X, 'test' : Xt, 'dev' : Xd}
labels = {'train' : y, 'test' : yt, 'dev' : yd}

import pandas as pd
df_texts = []
df_labels = []
df_exp_splits = []

for key in ['train', 'test', 'dev'] :
    df_texts += texts[key]
    df_labels += labels[key]
    df_exp_splits += [key] * len(texts[key])
    
data = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_splits})

In [4]:
import pandas as pd
import json

import os
os.makedirs('data', exist_ok=True)

In [5]:
for key in ['train', 'dev', 'test'] :
    data_key = data[data.exp_split == key].to_dict(orient='records')
    data_key = [{'document' : x['text'], 'label' : 'positive' if x['label'] == 1 else 'negative'} for x in data_key]
    f = open('data/' + key + '.jsonl', 'w')
    f.write('\n'.join([json.dumps(line) for line in data_key]))
    f.close()